# Variable Scope and Name Scope

** Author : Sangkeun Jung (hugmanskj@gmail.com)**

Tensorflow provides two different scope methods to group and specify variables, operations and constants. To understand it, it is better to check how tensorflow to name variables and operations first. 

## Tensorflow naming method

In [1]:
import tensorflow as tf
import numpy as np

C:\Users\juni\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Create variable with 'get_variable_scope'

- tf.get_variable(name, shape=None, dtype=None, initializer=None, regularizer=None, trainable=True, collections=None, caching_device=None, partitioner=None, validate_shape=True, custom_getter=None)
> Gets an existing variable with these parameters or create a new one.

In [2]:
a = tf.get_variable("a", [])
print(a.name)  ## name will be 'a:0' 

a:0


In [3]:
# create variable of same name
# it will generate errors!
try: 
    tf.get_variable("a", [])
except ValueError as E:
    print("ERROR!!!")
    print(E)

ERROR!!!
Variable a already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\juni\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1740, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Users\juni\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3414, in create_op
    op_def=op_def)
  File "C:\Users\juni\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)



## Variable Scope

In [4]:
with tf.variable_scope("A"):
    b = tf.get_variable("b", [])
    print(b.name) # --> A/b:0       A = scope name, b = variable name

A/b:0


With variable_scope, we can create many variables with same names.

In [5]:
with tf.variable_scope("C"):
    b2 = tf.get_variable("b", [])
    print(b2.name) # --> C/b:0       C = scope name, b = variable name

C/b:0


In [6]:
from graph_visualizer import show_graph
show_graph(tf.get_default_graph().as_graph_def())

## Nested variable scope

We can nest variable scope many times.

In [7]:
with tf.variable_scope("C"):
    with tf.variable_scope("D"):
        c = tf.get_variable("b", [])
        print(c)
        print(c.name) # --> C/D/b:0       C, D = scope name, b = variable name

<tf.Variable 'C/D/b:0' shape=() dtype=float32_ref>
C/D/b:0


## Reusing existing variable

We can reuse existing variable with get_variable_scope and 'reuse=True'.

In [8]:
with tf.variable_scope("C"):
    with tf.variable_scope("D", reuse=True):  # <--- Notice, 'reuse'
        alias_c = tf.get_variable("b")
        print(alias_c)
        print(alias_c.name)        

<tf.Variable 'C/D/b:0' shape=() dtype=float32_ref>
C/D/b:0


Or, we can set all variables in the scope will be 'reused' inside of the scope as follows:

In [9]:
with tf.variable_scope("C"):
    with tf.variable_scope("D"): # <-- note! no 'reuse' flag here!
        m = tf.get_variable("m", [])
        print m
        print m.name
        
        print "current variable scope :", tf.get_variable_scope().name
        tf.get_variable_scope().reuse_variables()
        m2 = tf.get_variable("m")
        print m2        
        print m2.name

C/D/m:0
current variable scope : C/D
C/D/m:0


## Namescope

- tf.name_scope(name, default_name=None, values=None)
> Returns a context manager for use when defining a Python op.

> This context manager validates that the given values are from the same graph, makes that graph the default graph, and pushes a name scope in that graph (see Graph.name_scope() for more details on that).

In [9]:
with tf.name_scope("custom_scope"):
    x = tf.get_variable("x", []) 
    y = tf.Variable(0.0, "x")
    op = x+y
    
print(x.name,  "  <--- tf.get_variable ignore name_scope")
print(y.name,  "  <--- name_scope() affects tf.Variable()")
print(op.name, "  <--  name_scope() affects operations")

x:0   <--- tf.get_variable ignore name_scope
custom_scope/Variable:0   <--- name_scope() affects tf.Variable()
custom_scope/add:0   <--  name_scope() affects operations


In [10]:
with tf.variable_scope("custom_scope2"):
    x = tf.get_variable("x", []) 
    y = tf.Variable(0.0, "y")
    op = x+y
    
print(x.name,  "  <--- variable_scope() affects tf.get_variable()")
print(y.name,  "  <--- variable_scope() affects tf.Variable()")
print(op.name, "  <--  variable_scope() affects operations")

custom_scope2/x:0   <--- variable_scope() affects tf.get_variable()
custom_scope2/Variable:0   <--- variable_scope() affects tf.Variable()
custom_scope2/add:0   <--  variable_scope() affects operations
